In [10]:

import gym
import gym_joinemio.envs.connect_four_env

env = gym.make('joinemio-v0')
iterations = 10
player1 = RandomPlayer()
player2 = RandomPlayer()

for i in range(iterations): # Iterate the game
    one_game = env.play_one_game(player1, player2, each_step_render=False)




# import gym
# import gym_joinemio
# from gym_joinemio.envs.player import RandomPlayer

# env = gym.make('joinemio-v0')
# iterations = 10
# player1 = RandomPlayer()
# player2 = RandomPlayer()

# for i in range(iterations): # Iterate the game
#     one_game = env.play_one_game(player1, player2, each_step_render=False)



[DEBUG][connect_four_env|gym_joinemio.envs.connect_four_env] Winner: 2
[DEBUG][connect_four_env|gym_joinemio.envs.connect_four_env] Winner: 2
[DEBUG][connect_four_env|gym_joinemio.envs.connect_four_env] Winner: 1
[DEBUG][connect_four_env|gym_joinemio.envs.connect_four_env] Winner: 2
[DEBUG][connect_four_env|gym_joinemio.envs.connect_four_env] Winner: 2
[DEBUG][connect_four_env|gym_joinemio.envs.connect_four_env] Winner: 1
[DEBUG][connect_four_env|gym_joinemio.envs.connect_four_env] Winner: 2
[DEBUG][connect_four_env|gym_joinemio.envs.connect_four_env] Winner: 1
[DEBUG][connect_four_env|gym_joinemio.envs.connect_four_env] Winner: 2
[DEBUG][connect_four_env|gym_joinemio.envs.connect_four_env] Winner: 2
